# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
# df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,<h1>Forbidden</h1><pre>Request ID: 2022-09-25-10-38-24-16BFC82C61131161</pre>


In [11]:
import re
from typing import Union, Callable

import pandas as pd

In [16]:
def main() -> None:
    data = pd.read_csv("water_spring_data.csv", sep=';')
    data.set_index('Показатель', inplace=True)
    data['Вывод'] = data.apply(check_norm, axis=1)
    print(data['Вывод'])


# Решил немного побаловаться декоратором
def df_wrapper(value_name: str, norm_name: str) -> Callable:

    def decorator(func: Callable) -> Callable:

        def wrapper(x: pd.DataFrame) -> Callable:
            return func(x[value_name], x[norm_name])

        return wrapper

    return decorator


@df_wrapper('Результат анализа', 'Норматив')
def check_norm(value: Union[str, int, float], norm: str) -> str:
    try:
        value = float(value)
    except ValueError:
        value = 0.0

    l_e = re.compile(r'не\s*бол[а-я]+', re.IGNORECASE)
    btw = re.compile(r'в\s*предел[а-я]+', re.IGNORECASE)
    dgts = re.compile(r'[0-9,]+')

    less_or_equal = l_e.search(norm)
    between = btw.search(norm)
    digits = list(map(parse_rus_digits, dgts.findall(norm)))

    if less_or_equal and digits:
        if value <= min(digits):
            return 'Показатель в норме'
        return 'Показатель не в норме'

    if between and digits:
        if min(digits) <= value <= max(digits):
            return 'Показатель в норме'
        return 'Показатель не в норме'

    return 'Не удалось определить'


def parse_rus_digits(s_digit: str) -> float:
    return float(s_digit.replace(',', '.'))

In [17]:
main()

Показатель
pH                                          Показатель в норме
Запах                                       Показатель в норме
Цветность                                   Показатель в норме
Жёсткость                                   Показатель в норме
Аммиак и аммоний-ион (по азоту)             Показатель в норме
Нитриты (по NO2)                            Показатель в норме
Нитраты (по NO3)                            Показатель в норме
Фосфаты (P)                                 Показатель в норме
Хлориды (Cl)                                Показатель в норме
Сульфаты (SO4)                              Показатель в норме
Железо (включая хлорное железо) по Fe       Показатель в норме
Нефть                                    Показатель не в норме
Общая минерализация (сухой остаток)         Показатель в норме
Окисляемость перманганатная              Показатель не в норме
Name: Вывод, dtype: object


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [35]:
from random import choice

n_expertiments = 100_000
res = 0
for _ in range(n_expertiments):
    box = []
    box.extend(['orange'] * 5)
    box.extend(['apple'] * 4)
    drafts = []
    for __ in range(3):
        draft = choice(box)
        drafts.append(draft)
        box.pop(box.index(draft))
    if 'apple' not in drafts:
        res += 1
print(res / n_expertiments)

0.11973


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [37]:
n_expertiments = 100_000
res = 0
for _ in range(n_expertiments):
    box = []
    box.extend(['standard'] * 7)
    box.extend(['non-standard'] * 3)
    draft = None
    count = 0
    while draft != 'standard':
        count += 1
        draft = choice(box)
        box.pop(box.index(draft))
    if count == 2:
        res += 1
print(res / n_expertiments)

0.2328
